In [1]:
import cv2
import mediapipe as mp
import csv
import pandas as pd
import os
import numpy as np

In [2]:
# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [3]:
#DATASET_DIR = 'Single_person_violent'
DATASET_DIR = 'Final_Dataset'
CLASSES_LIST = ["Kicking","Punching","Block","Idle"]
OUTPUT_DIR = 'Output'
custom_headers = ['Frame Number', 'x','y','z','Visibility']

In [4]:
# Convert mediapipe landmarks into proper format for storing into a CSV file
def write_landmarks_to_csv(landmarks, frame_number, csv_data):
    #print(f"Landmark coordinates for frame {frame_number}:")
    for idx, landmark in enumerate(landmarks):
        #print(f"{mp_pose.PoseLandmark(idx).name}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z})")
        csv_data.append([frame_number, landmark.x, landmark.y, landmark.z, landmark.visibility])    

In [8]:
# Convert a video into landmarks using mediapipe
def convert_video_to_landmark_csv(video_path):
    frame_number = 0
    csv_data = []

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():            
            ret, frame = cap.read()
            if not ret:
                break
            frame_number += 1
            # Convert the image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Perform pose detection
            results = pose.process(image)

            # Convert the image back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw the pose annotation on the image
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Add the landmark coordinates to the list and print them
            write_landmarks_to_csv(results.pose_landmarks.landmark, frame_number, csv_data)   
    cap.release() 

    # Extract file and class names from the video path
    file_name = os.path.splitext(os.path.basename(video_path))[0]
    class_name = os.path.basename(os.path.dirname(video_path))

    
    
    # Ensure the output directory exists
    output_file_dir = os.path.join(OUTPUT_DIR, class_name)        
    os.makedirs(output_file_dir, exist_ok=True)  

    output_file = os.path.join(output_file_dir, f'{file_name}.csv')

    with open(f'{output_file}.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write each row of the 2D array
        for row in csv_data:
            writer.writerow(row)

In [6]:
# Convert all videos in the given CLASSES_LIST into landmark csv files
def create_dataset(CLASSES_LIST):
    print('List: ',CLASSES_LIST)
    for class_name in CLASSES_LIST:
        print(f"Extracting data from {class_name}")
        # Get list of videos for each class
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        for file_name in files_list:
            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
            convert_video_to_landmark_csv(video_file_path)

In [57]:
# Create a sequence from a CSV file for making a single dataframe
def createSequence(csv_path, label):
    # Define custom headers for the CSV file
    custom_headers = ['Frame Number', 'x', 'y', 'z', 'Visibility']
    
    # Read the CSV file into a DataFrame
    data = pd.read_csv(csv_path, header=None, names=custom_headers)
    
    # Initialize the sequence with the label
    sequence = [label]
    
    # Group the data by 'Frame Number' and collect frame data
    grouped = data.groupby('Frame Number')[['x', 'y', 'z', 'Visibility']].apply(lambda x: x.values.tolist())
    
    # Extend the sequence with the grouped frame data
    sequence.extend(grouped.tolist())
    
    return sequence


In [58]:
# Read data from CSV and append to processed_df
sequence = createSequence('output.csv','Kicking')
processed_df = pd.DataFrame([sequence])
#processed_df2 = processed_df.append(pd.DataFrame(frame, columns=['x', 'y', 'z', 'Visibility']), ignore_index=True)
processed_df.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,Kicking,"[[0.4208270311355591, 0.2813926935195923, -0.5...","[[0.4226616024971008, 0.2830023467540741, -0.4...","[[0.422335147857666, 0.2832168936729431, -0.48...","[[0.4239648282527923, 0.2812423706054687, -0.2...","[[0.4259682595729828, 0.2790455818176269, -0.2...","[[0.424310564994812, 0.2764437794685364, -0.26...","[[0.424225777387619, 0.2755632400512695, 0.049...","[[0.4241669476032257, 0.2759043276309967, -0.1...","[[0.4190621674060821, 0.2758240699768066, -0.0...",...,"[[0.4902646839618683, 0.266271561384201, -0.52...","[[0.4924321174621582, 0.2656450271606445, -0.5...","[[0.4942595660686493, 0.2644723653793335, -0.5...","[[0.4932672381401062, 0.2639179527759552, -0.5...","[[0.4929601550102234, 0.264041006565094, -0.53...","[[0.4959222972393036, 0.2640158534049988, -0.5...","[[0.5004463791847229, 0.2640080451965332, -0.5...","[[0.5007750391960144, 0.2638412415981293, -0.4...","[[0.4997018575668335, 0.2627485990524292, -0.4...","[[0.4992328286170959, 0.262261152267456, -0.44..."


In [64]:
dat = pd.read_csv('output.csv',header=None,names=custom_headers)
dat['Frame Number'].tail(1).values[0]
#print(dat.groupby('FrameNumber')['x'].apply(lambda x: x.tolist()).iloc[:1][1])

37

In [13]:
# Get all the CSV files of the given CLASSES_LIST
def getAllCSV(CLASSES_LIST):
    files = {}
    for class_name in CLASSES_LIST:        
        # Get list of csv files for each class
        files_list = os.listdir(os.path.join(OUTPUT_DIR, class_name))
        for file_name in files_list:
            # Get the complete csv path.
            files.setdefault(class_name,[]).append(os.path.join(OUTPUT_DIR, class_name, file_name))
    return files

In [14]:
# Merge all the sequences into a single dataframe and return it
def createDataframe():
    df_array = []    
    for class_name in CLASSES_LIST:        
        # Get list of csv files for each class
        files_list = os.listdir(os.path.join(OUTPUT_DIR, class_name))
        for file_name in files_list:
            # Get the complete csv path.
            csv_file_path = os.path.join(OUTPUT_DIR, class_name, file_name)            
            df_array.append(createSequence(csv_file_path,class_name))
    return pd.DataFrame(df_array)
            

In [15]:
# Calculate the mean and standard deviation of z values (depth values)
def calculateZParamsForNormalization(CLASSES_LIST,csv_files):
    custom_headers = ['FrameNumber','x','y','z','Visibility']
    z_values = []
    for key in CLASSES_LIST:
        for file in csv_files[key]:
            df = pd.read_csv(file,header=None,names=custom_headers)
            z_values.extend(df['z'].values)
    z_mean = np.mean(z_values)
    z_std = np.std(z_values)
    return z_mean, z_std

In [16]:
# Calculate the min and max of z values (depth values)
def calculateZMinMaxForNormalization(CLASSES_LIST,csv_files):
    custom_headers = ['FrameNumber','x','y','z','Visibility']
    z_values = []
    for key in CLASSES_LIST:
        for file in csv_files[key]:
            df = pd.read_csv(file,header=None,names=custom_headers)
            z_values.extend(df['z'].values)
    z_min = np.min(z_values)
    z_max = np.max(z_values)
    return z_min, z_max

In [28]:
# Normalize all the z values (depth values) in the CSV files
def normalizeZValuesInCSV(CLASSES_LIST):
    custom_headers = ['FrameNumber','x','y','z','Visibility']
    files = getAllCSV(CLASSES_LIST)
    z_min, z_max = calculateZMinMaxForNormalization(CLASSES_LIST,files)
    for key in files:
        for file in files[key]:
            df = pd.read_csv(file,header=None,names=custom_headers)
            # Normalize the 'z' column using Min-Max scaling
            df['z'] = (df['z'] - z_min) / (z_max - z_min)
            df.to_csv(file, index=False, header=False)

In [72]:
def findHighestFrameNumber(CLASSES_LIST):
    highest_frame_number = 0
    custom_headers = ['FrameNumber','x','y','z','Visibility']
    files = getAllCSV(CLASSES_LIST)        
    highest_frame_video = ''
    for key in files:        
        for file in files[key]:            
            df = pd.read_csv(file,header=None,names=custom_headers)
            current_frame_number = df['FrameNumber'].tail(1).values[0]
            if  current_frame_number > highest_frame_number:
                highest_frame_number = current_frame_number  
                highest_frame_video  = file      
    return highest_frame_number,highest_frame_video

In [77]:
def findFramesHigherThanX(CLASSES_LIST,X):    
    custom_headers = ['FrameNumber','x','y','z','Visibility']
    files = getAllCSV(CLASSES_LIST)        
    high_frame_videos = []
    for key in files:        
        for file in files[key]:            
            df = pd.read_csv(file,header=None,names=custom_headers)
            current_frame_number = df['FrameNumber'].tail(1).values[0]
            if  current_frame_number > X:
                high_frame_videos.append((file,current_frame_number))
    return high_frame_videos

In [9]:
create_dataset(CLASSES_LIST)

List:  ['Kicking', 'Punching', 'Block', 'Idle']
Extracting data from Kicking
Extracting data from Punching
Extracting data from Block
Extracting data from Idle


In [29]:
normalizeZValuesInCSV(CLASSES_LIST)

In [80]:
high_frame_videos = findFramesHigherThanX(CLASSES_LIST,100)
high_frame_videos

[('Output\\Kicking\\kick_left13.csv', 130),
 ('Output\\Kicking\\kick_right2.csv', 102),
 ('Output\\Block\\block6.csv', 110),
 ('Output\\Idle\\idle16.csv', 216),
 ('Output\\Idle\\idle17.csv', 210),
 ('Output\\Idle\\idle4.csv', 107)]

In [73]:
highestFrameNumber, highestFrameVideo = findHighestFrameNumber(CLASSES_LIST)            

In [74]:
highestFrameNumber, highestFrameVideo

(216, 'Output\\Idle\\idle16.csv')

In [ ]:
complete_df = createDataframe()

In [ ]:
complete_df[complete_df.iloc[:,0] == 'Punching'][:5]

In [31]:
files = getAllCSV(CLASSES_LIST)

In [ ]:
df2 = pd.read_csv('output.csv',header=None,names=custom_headers)
arr = []
arr.extend(df2['z'].values)
arr

In [33]:
z_mean, z_std = calculateZParamsForNormalization(CLASSES_LIST,files)

In [36]:
z_min, z_max = calculateZMinMaxForNormalization(CLASSES_LIST,files)

In [34]:
z_mean, z_std
#(-0.10812381639618134, 0.44725407070063944)

(0.5545347004295016, 0.09994274934490296)

In [37]:
z_min, z_max
#(-1.6407425403594973, 1.754867672920227)

(0.0, 1.0)

In [50]:
for class_name in CLASSES_LIST:        
    # Get list of csv files for each class
    files_list = os.listdir(os.path.join(OUTPUT_DIR, class_name))    
    for filename in files_list:
        if filename.endswith('.csv.csv'):
            folder_name = os.path.join(OUTPUT_DIR,class_name)
            # Construct the new filename by removing the extra .csv
            new_filename = os.path.join(folder_name, filename[:-4])
            # Rename the file
            os.rename(os.path.join(folder_name, filename), new_filename)
            print(f'Renamed {filename} to {filename[:-4]}')



Renamed kick16a.csv.csv to kick16a.csv
Renamed kick16b.csv.csv to kick16b.csv
Renamed kick17a.csv.csv to kick17a.csv
Renamed kick17b.csv.csv to kick17b.csv
Renamed kick18a.csv.csv to kick18a.csv
Renamed kick18b.csv.csv to kick18b.csv
Renamed kick19a.csv.csv to kick19a.csv
Renamed kick19b.csv.csv to kick19b.csv
Renamed kick20a.csv.csv to kick20a.csv
Renamed kick20b.csv.csv to kick20b.csv
Renamed kick21a.csv.csv to kick21a.csv
Renamed kick21b.csv.csv to kick21b.csv
Renamed kick22a.csv.csv to kick22a.csv
Renamed kick22b.csv.csv to kick22b.csv
Renamed kick23a.csv.csv to kick23a.csv
Renamed kick23b.csv.csv to kick23b.csv
Renamed kick24a.csv.csv to kick24a.csv
Renamed kick24b.csv.csv to kick24b.csv
Renamed kick25a.csv.csv to kick25a.csv
Renamed kick25b.csv.csv to kick25b.csv
Renamed kick26a.csv.csv to kick26a.csv
Renamed kick26b.csv.csv to kick26b.csv
Renamed kick27a.csv.csv to kick27a.csv
Renamed kick27b.csv.csv to kick27b.csv
Renamed kick28a.csv.csv to kick28a.csv
Renamed kick28b.csv.csv t